<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instal and Setup (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

#### Install Packages

In [49]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-a14_tw9t
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-a14_tw9t
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 6196a4d3d4526c127c2b6d7896414206d9f9267c
  Preparing metadata (setup.py) ... done


In [50]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [51]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [52]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='ibd'
)

In [53]:
# @title DataFrame Operations
import vistock.tw as tw

def add_price_column(df):
    column_names = df.columns.tolist()
    if 'Price' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    if not ticker.isdigit():    # if not a Taiwan stock
        return df
    df['Price'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Price')
    df = df[column_names]
    df['Price'] = df['Ticker'].apply(tw.stock_price)
    return df

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    if not ticker.isdigit():    # if not a Taiwan stock
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df

def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]

### Glossary of Terms

source (Select the source of stocks to analyze):
- The source of stocks to analyze
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange
    - `TPEX`: Taipei Exchange
    - `ESB`: Emerging Stock Board
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500
    - `DJIA`: Dow Jones Industrial Average
    - `NDX`: NASDAQ-100
    - `SOX`: PHLX Semiconductor Index
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.


### Execute Actions Step by Step


In [54]:
# @title Step 1. Pick a File

import ipywidgets as widgets
from IPython.display import display

# List of filenames
filenames = github.list_filenames()

# Extract and sort all unique values
dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames for setting dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px per character for width

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create the file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

# Function to update file options based on selected filters
def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value in fn and
            type_dropdown.value in fn and
            period_dropdown.value in fn)
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match

# Function to handle file selection and print the selected filename
#def on_file_selected(change):
#    selected_file = change['new']
#    print(f"Selected file: {selected_file}")

# Update file options when any of the dropdowns change
date_dropdown.observe(update_file_options, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Print the selected file when it changes
#file_dropdown.observe(on_file_selected, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize the file dropdown with options
update_file_options()


Dropdown(description='Date:', layout=Layout(width='430px'), options=('20240812', '20240811', '20240809'), valu…

Dropdown(description='Source:', layout=Layout(width='430px'), options=('TWSE+TPEX', 'SPX+DJIA+NDX+SOX'), value…

Dropdown(description='Type:', layout=Layout(width='430px'), options=('stocks', 'industries'), value='stocks')

Dropdown(description='Period:', layout=Layout(width='430px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='430px'), options=(), value=None)

In [55]:
# @title Step 2. Print Top Percentile Items {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
df_top = df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: TWSE+TPEX
Type: stocks
Period: 2y
Date: 20240812


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,6144,得利影,84.8,Communication Services,Entertainment,460.73,269.70,117.46,84.28,99.0,99,87,19,1
1,1799,易威,203.5,Healthcare,Drug Manufacturers—Specialty & Generic,435.74,370.36,208.83,203.81,99.0,99,99,98,2
2,2365,昆盈,62.9,Technology,Computer Hardware,354.58,302.11,127.65,193.74,99.0,99,91,98,3
3,8374,羅昇,133.0,Industrials,Industrial Distribution,349.67,277.16,84.36,82.91,99.0,99,43,16,4
4,6442,光聖,342.5,Technology,Electronic Components,325.35,449.05,198.94,164.48,99.0,99,98,96,5
5,6640,均華,687.0,Technology,Semiconductors,314.97,307.41,236.56,179.45,99.0,99,99,97,6
6,4562,穎漢,76.2,Industrials,Specialty Industrial Machinery,313.84,301.56,153.11,84.69,99.0,99,95,20,7
7,2543,皇昌,62.5,Industrials,Engineering & Construction,292.97,280.49,280.57,242.42,99.0,99,99,99,8
8,2348,海悅,264.0,Real Estate,Real Estate Services,284.76,232.88,205.92,120.24,99.0,99,99,85,9
9,4909,新復興,120.0,Technology,Electronic Components,283.82,259.78,246.93,225.51,99.0,99,99,99,10


['得利影', '易威', '昆盈', '羅昇', '光聖', '均華', '穎漢', '皇昌', '海悅', '新復興']
['6144', '1799', '2365', '8374', '6442', '6640', '4562', '2543', '2348', '4909']


In [56]:
# @title Step 3. Filter Items with Increasing RS > 100
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
filtered_df = filter_increasing_relative_strength(df)
df_top = filtered_df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: TWSE+TPEX
Type: stocks
Period: 2y
Date: 20240812


,Ticker,Name,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,6144,得利影,84.8,Communication Services,Entertainment,460.73,269.70,117.46,84.28,99.0,99,87,19,1
1,1799,易威,203.5,Healthcare,Drug Manufacturers—Specialty & Generic,435.74,370.36,208.83,203.81,99.0,99,99,98,2
3,8374,羅昇,133.0,Industrials,Industrial Distribution,349.67,277.16,84.36,82.91,99.0,99,43,16,4
5,6640,均華,687.0,Technology,Semiconductors,314.97,307.41,236.56,179.45,99.0,99,99,97,6
6,4562,穎漢,76.2,Industrials,Specialty Industrial Machinery,313.84,301.56,153.11,84.69,99.0,99,95,20,7
8,2348,海悅,264.0,Real Estate,Real Estate Services,284.76,232.88,205.92,120.24,99.0,99,99,85,9
9,4909,新復興,120.0,Technology,Electronic Components,283.82,259.78,246.93,225.51,99.0,99,99,99,10
10,9906,欣巴巴,199.0,Industrials,Engineering & Construction,273.25,170.24,119.85,109.67,99.0,97,89,78,11
16,2524,京城,108.0,Real Estate,Real Estate—Development,243.24,153.97,136.39,86.42,99.0,96,94,27,17
26,1225,福懋油,153.0,Consumer Defensive,Packaged Foods,216.29,180.07,106.89,88.76,98.0,97,81,35,27


['得利影', '易威', '羅昇', '均華', '穎漢', '海悅', '新復興', '欣巴巴', '京城', '福懋油']
['6144', '1799', '8374', '6640', '4562', '2348', '4909', '9906', '2524', '1225']
